<a href="https://colab.research.google.com/github/Abubakrhegazi/Arabic-Grammar-AutoCorrection/blob/main/Arabic_GED_GEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
# prompt: import from drive folder named grammar and read an m2 file

from google.colab import drive
drive.mount('/content/drive')

# file_path = '/content/drive/My Drive/grammar/QALB-Train2014.m2'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [89]:
#functions dictionary
def smart_open(fname, mode = 'r'):
    if fname.endswith('.gz'):
        import gzip
        # Using max compression (9) by default seems to be slow.
        # Let's try using the fastest.
        return gzip.open(fname, mode, 1)
    else:
        return open(fname, mode)

import sys
from getopt import getopt

def load_annotation(gold_file):
    source_sentences = []
    gold_edits = []
    fgold = smart_open(gold_file, 'r')
    puffer = fgold.read()
    fgold.close()
    #puffer = puffer.decode('utf8')
    for item in paragraphs(puffer.splitlines(True)):
        item = item.splitlines(False)
        sentence = [line[2:].strip() for line in item if line.startswith('S ')]
        assert sentence != []
        annotations = {}
        for line in item[1:]:
            if line.startswith('I ') or line.startswith('S '):
                continue
            assert line.startswith('A ')
            line = line[2:]
            fields = line.split('|||')
            #print(fields)
            start_offset = int(fields[0].split()[0])
            end_offset = int(fields[0].split()[1])
            etype = fields[1]
            if etype == 'noop':
                start_offset = -1
                end_offset = -1
            corrections =  [c.strip() if c != '-NONE-' else '' for c in fields[2].split('||')]
            # NOTE: start and end are *token* offsets
            original = ' '.join(' '.join(sentence).split()[start_offset:end_offset])
            if len(fields) > 4:
              annotator = int(fields[5])
            if annotator not in annotations.keys():
                annotations[annotator] = []
            annotations[annotator].append((start_offset, end_offset, original, corrections))
        tok_offset = 0
        for this_sentence in sentence:
            tok_offset += len(this_sentence.split())
            source_sentences.append(this_sentence)
            this_edits = {}
            for annotator, annotation in annotations.items():
                this_edits[annotator] = [edit for edit in annotation if edit[0] <= tok_offset and edit[1] <= tok_offset and edit[0] >= 0 and edit[1] >= 0]
            if len(this_edits) == 0:
                this_edits[0] = []
            gold_edits.append(this_edits)
    return (source_sentences, gold_edits)
def paragraphs(lines, is_separator=lambda x : x == '\n', joiner=''.join):
    paragraph = []
    for line in lines:
        if is_separator(line):
            if paragraph:
                yield joiner(paragraph)
                paragraph = []
        else:
            paragraph.append(line)
    if paragraph:
        yield joiner(paragraph)

In [90]:
m2_list_sentence=[]
m2_list_operation=[]

m2_list_sentence.append(load_annotation('/content/drive/MyDrive/Grammar/QALB-Train2014.m2')[0])
m2_list_operation.append(load_annotation('/content/drive/MyDrive/Grammar/QALB-Test2014.m2')[1])


In [91]:
# For extracting actual corrections

def extract(operation):
  input=[]
  correct=[]

  for i in range(len(operation)):
          if (operation[i][0]!=operation[i][1]):
              input.append(operation[i][2])
              correct.append(operation[i][3][0])
  return (input , correct)

In [92]:
def make_tokens(list):
  tokens =[]
  for s in list:
      for i in s:
          tokens.append(i.split(' '))
  return tokens


In [93]:
tokens =[]
i=0
for i in m2_list_sentence:
    tokens = make_tokens(m2_list_sentence)
# print(tokens[0])
# len(tokens)

In [94]:
print("len(tokens):", len(tokens))
print("len(m2_list_operation[0]):", len(m2_list_operation[0]))


len(tokens): 19411
len(m2_list_operation[0]): 968


In [95]:
list_correction = []
list_input = []

for i in range(len(m2_list_operation[0])):  # ✅ Use the shorter list
    edits = m2_list_operation[0][i][0]
    input_, correction = extract(edits)
    list_input.append(input_)
    list_correction.append(correction)


In [96]:
for i in range(5):  # test like df.head f pandas
    print(f"Input {i}:", list_input[i])
    print(f"Correction {i}:", list_correction[i])
    print()

Input 0: ['لا زال', 'الشبيحه', 'ان', 'ارواح', 'اقل', 'كلفه', 'اذا', 'المعادله', 'المهينه', 'ان', 'هبه', 'قويه', 'واحده', 'وياخذو', 'العصابه', 'عنوه', '،', 'اننا', 'اكثر', 'اربعين', 'عام']
Correction 0: ['لازال', 'الشبيحة', 'أن', 'أرواح', 'أقل', 'كلفة', 'إذا', 'المعادلة', 'المهينة', 'أن', 'هبة', 'قوية', 'واحدة', 'ويأخذوا', 'العصابة', 'عنوة', '.', 'إننا', 'أكثر', 'أربعين', 'عاما']

Input 1: ['امريكان', 'و يا', 'اذنابهم', 'ان', 'امن', 'تستطيعوا', 'لان', 'و سترون', 'الاسد', 'الاكتاف', 'تهو', 'الامويين']
Correction 1: ['أمريكان', 'ويا', 'أذنابهم', 'أن', 'أمن', 'تستطيعون', 'لأن', 'وسترون', 'الأسد', 'الأكتاف', 'تهز', 'الأمويين']

Input 2: ['إعتقاد']
Correction 2: ['اعتقاد']

Input 3: ['راينا', 'اماكن', 'و للتقطيع', 'بالتاكيد', 'او', 'او', 'ابيا', 'واسفاه', 'و ساند', 'اذيالها']
Correction 3: ['رأينا', 'أماكن', 'وللتقطيع', 'بالتأكيد', 'أو', 'أو', 'أبيا', 'وأسفاه', 'وساند', 'أذيالها']

Input 4: ['الى', 'مشاركه', 'انت', 'الاحرار', 'لايخافون', 'لانهم', 'الانفس', 'البريئه', 'اما', 'الارانب', 'لانه'

In [97]:
# Join tokens back into strings
inputs = [' '.join(words) for words in list_input]
corrections = [' '.join(words) for words in list_correction]


In [98]:
import pandas as pd

df = pd.DataFrame({
    'input': [' '.join(x) for x in list_input],
    'correction': [' '.join(y) for y in list_correction]
})
df.to_csv('grammar_corrections.csv', index=False)
df.head(10)

,input,correction
0,لا زال الشبيحه ان ارواح اقل كلفه اذا المعادله ...,لازال الشبيحة أن أرواح أقل كلفة إذا المعادلة ا...
1,امريكان و يا اذنابهم ان امن تستطيعوا لان و ستر...,أمريكان ويا أذنابهم أن أمن تستطيعون لأن وسترون...
2,إعتقاد,اعتقاد
3,راينا اماكن و للتقطيع بالتاكيد او او ابيا واسف...,رأينا أماكن وللتقطيع بالتأكيد أو أو أبيا وأسفا...
4,الى مشاركه انت الاحرار لايخافون لانهم الانفس ا...,إلى مشاركة أنت الأحرار لا يخافون لأنهم الأنفس ...
5,,
6,اخر ايامك احرار الا ان تكبرو ان اتي احرار صلو ...,آخر أيامك أحرار إلا أن تكبروا إن أتى أحرار صلو...
7,"اثبت انه اسد و في , اوقف الاسرائيلي سوري فماكا...",أثبت أنه أسد وفي ، أوقف الإسرائيلي سورية فما ك...
8,الاطفال ارهابيون لانهم الامة الامة الى اخر الا...,الأطفال إرهابيون لأنهم الأمة الأمة إلى آخر ألا...
9,احبكم ارض امام اجندة إحتلال وامام انا إبن إبن,أحبكم أرض أمام أجندة احتلال وأمام أنا ابن ابن


In [99]:
# Check for nulls in the DataFrame
print(df.isnull().sum())

input         0
correction    0
dtype: int64


In [100]:
# Preprocessing regular expressions
import re
def process(sent):
    # sent=sent.lower()
    # sent=re.sub(r'[^0-9]','',sent)

    #sent=sent.replace(' ','\n')
    sent=sent.replace('ـ','')
    sent=sent.replace('-','')
    sent=sent.replace(')','')
    sent=sent.replace('(','')
    sent=sent.replace('*','')
    sent=sent.replace(',','')
    sent=sent.replace('/','')
    sent=sent.replace('&','')
    sent=sent.replace('amp','')
    sent=sent.replace('_','')
    sent=sent.replace('ّ','')
    sent=sent.replace('ً','')
    sent=sent.replace('%','')
    sent=sent.replace(';','')
    sent=sent.replace('ِ','')
    sent=sent.replace('@','')
    sent=sent.replace('ګ','')
    sent=sent.replace( 'ٍ','')
    sent=sent.replace('٪','')
    sent=sent.replace('ٱ','')
    sent=sent.replace('ُ','')
    sent=sent.replace(':','')
    sent=sent.replace('؛','')
    sent=sent.replace('[','')
    sent=sent.replace(']','')
    sent=sent.replace('=','')
    sent=sent.replace('^','')
    sent=sent.replace('?','')
    sent=sent.replace('{','')
    sent=sent.replace('}','')
    sent=sent.replace('g','')
    sent=sent.replace('t','')
    sent=sent.replace('a','')
    sent=sent.replace('b','')
    sent=sent.replace('c','')
    sent=sent.replace('d','')
    sent=sent.replace('s','')
    sent=sent.replace('f','')
    sent=sent.replace('h','')
    sent=sent.replace('j','')
    sent=sent.replace('k','')
    sent=sent.replace('l','')
    sent=sent.replace('q','')
    sent=sent.replace('t','')
    sent=sent.replace('a','')
    sent=sent.replace('b','')
    sent=sent.replace('c','')
    sent=sent.replace('d','')
    sent=sent.replace('s','')
    sent=sent.replace('f','')
    sent=sent.replace('h','')
    sent=sent.replace('j','')
    sent=sent.replace('k','')
    sent=sent.replace('l','')
    sent=sent.replace('q','')
    sent=sent.replace('w','')
    sent=sent.replace('e','')
    sent=sent.replace('r','')
    sent=sent.replace('y','')
    sent=sent.replace('u','')
    sent=sent.replace('i','')
    sent=sent.replace('o','')
    sent=sent.replace('p','')
    sent=sent.replace('z','')
    sent=sent.replace('x','')
    sent=sent.replace('v','')
    sent=sent.replace('n','')
    sent=sent.replace('m','')
    sent=sent.replace('\\','')
    sent=sent.replace('$','')
    sent=sent.replace('`','')
    sent=sent.replace('~','')
    sent=sent.replace('|','')
    sent=sent.replace('گ','')
    sent=sent.replace("'",'')
    # sent=sent.replace('ُ','')
    # sent=sent.replace(':','')
    # sent=sent.replace('ُ','')
    # sent=sent.replace(':','')
    # sent=sent.replace('ُ','')
    # sent=sent.replace(':','')
    # sent=sent.replace('ُ','')
    # sent=sent.replace(':','')
    # sent=sent.replace('ُ','')
    # sent=sent.replace(':','')

    return sent

In [101]:
# df['processed_input'] = df['input'].apply(process)
# df['processed_correction'] = df['correction'].apply(process)
# df.head()

In [102]:
import re

def standerdize(text):
    # Define characters to remove
    text = re.sub(r'[a-zA-Zـًٍُِّٱ٪\*\,/\\:\؛@\$&_\^~\|\'"=\?\{\}\[\]\(\)`]', '', text)
    return text.strip()


In [103]:
raw_text = "ـأحبكم @أرض أمام أجندة amp إحتلال، وأمام أنا إبن إبنً!"

print(raw_text)
cleaned = standerdize(raw_text)
print(cleaned)
proc = process(raw_text)
print(proc)

ـأحبكم @أرض أمام أجندة amp إحتلال، وأمام أنا إبن إبنً!
أحبكم أرض أمام أجندة  إحتلال، وأمام أنا إبن إبن!
أحبكم أرض أمام أجندة  إحتلال، وأمام أنا إبن إبن!


In [104]:
df.head()

,input,correction
0,لا زال الشبيحه ان ارواح اقل كلفه اذا المعادله ...,لازال الشبيحة أن أرواح أقل كلفة إذا المعادلة ا...
1,امريكان و يا اذنابهم ان امن تستطيعوا لان و ستر...,أمريكان ويا أذنابهم أن أمن تستطيعون لأن وسترون...
2,إعتقاد,اعتقاد
3,راينا اماكن و للتقطيع بالتاكيد او او ابيا واسف...,رأينا أماكن وللتقطيع بالتأكيد أو أو أبيا وأسفا...
4,الى مشاركه انت الاحرار لايخافون لانهم الانفس ا...,إلى مشاركة أنت الأحرار لا يخافون لأنهم الأنفس ...


In [105]:
df['input'] = df['input'].apply(standerdize)
df['correction_new'] = df['correction'].apply(standerdize)
df['correction_old'] = df['correction'].apply(process)
df.head(20)

,input,correction,correction_new,correction_old
0,لا زال الشبيحه ان ارواح اقل كلفه اذا المعادله ...,لازال الشبيحة أن أرواح أقل كلفة إذا المعادلة ا...,لازال الشبيحة أن أرواح أقل كلفة إذا المعادلة ا...,لازال الشبيحة أن أرواح أقل كلفة إذا المعادلة ا...
1,امريكان و يا اذنابهم ان امن تستطيعوا لان و ستر...,أمريكان ويا أذنابهم أن أمن تستطيعون لأن وسترون...,أمريكان ويا أذنابهم أن أمن تستطيعون لأن وسترون...,أمريكان ويا أذنابهم أن أمن تستطيعون لأن وسترون...
2,إعتقاد,اعتقاد,اعتقاد,اعتقاد
3,راينا اماكن و للتقطيع بالتاكيد او او ابيا واسف...,رأينا أماكن وللتقطيع بالتأكيد أو أو أبيا وأسفا...,رأينا أماكن وللتقطيع بالتأكيد أو أو أبيا وأسفا...,رأينا أماكن وللتقطيع بالتأكيد أو أو أبيا وأسفا...
4,الى مشاركه انت الاحرار لايخافون لانهم الانفس ا...,إلى مشاركة أنت الأحرار لا يخافون لأنهم الأنفس ...,إلى مشاركة أنت الأحرار لا يخافون لأنهم الأنفس ...,إلى مشاركة أنت الأحرار لا يخافون لأنهم الأنفس ...
5,,,,
6,اخر ايامك احرار الا ان تكبرو ان اتي احرار صلو ...,آخر أيامك أحرار إلا أن تكبروا إن أتى أحرار صلو...,آخر أيامك أحرار إلا أن تكبروا إن أتى أحرار صلو...,آخر أيامك أحرار إلا أن تكبروا إن أتى أحرار صلو...
7,اثبت انه اسد و في اوقف الاسرائيلي سوري فماكان...,أثبت أنه أسد وفي ، أوقف الإسرائيلي سورية فما ك...,أثبت أنه أسد وفي ، أوقف الإسرائيلي سورية فما ك...,أثبت أنه أسد وفي ، أوقف الإسرائيلي سورية فما ك...
8,الاطفال ارهابيون لانهم الامة الامة الى اخر الا...,الأطفال إرهابيون لأنهم الأمة الأمة إلى آخر ألا...,الأطفال إرهابيون لأنهم الأمة الأمة إلى آخر ألا...,الأطفال إرهابيون لأنهم الأمة الأمة إلى آخر ألا...
9,احبكم ارض امام اجندة إحتلال وامام انا إبن إبن,أحبكم أرض أمام أجندة احتلال وأمام أنا ابن ابن,أحبكم أرض أمام أجندة احتلال وأمام أنا ابن ابن,أحبكم أرض أمام أجندة احتلال وأمام أنا ابن ابن


In [106]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)

https://docs.google.com/spreadsheets/d/18UCE_CEtBVw1cDDV2GsJ0YHyY1o-XAEX12ZIPXJWgsY/edit#gid=0


In [107]:
# Find rows where 'input' column only contains whitespace
rows_with_only_space = df[df['input'].str.match(r'^\s*$')]

# Print the rows that match the condition
print("Rows with only whitespace in 'input' column:")
rows_with_only_space

Rows with only whitespace in 'input' column:


,input,correction,correction_new,correction_old
5,,,,
25,,,,
79,,,,
95,,,,
241,,,,
336,,,,
365,,,,
455,,،,،,،
494,,,,
594,,,,


In [108]:
# Drop rows where 'input' column only contains whitespace
df = df[~df['input'].str.match(r'^\s*$')]
print("\nDataFrame after dropping rows with only whitespace in 'input' column:")
df.head(10)


DataFrame after dropping rows with only whitespace in 'input' column:


,input,correction,correction_new,correction_old
0,لا زال الشبيحه ان ارواح اقل كلفه اذا المعادله ...,لازال الشبيحة أن أرواح أقل كلفة إذا المعادلة ا...,لازال الشبيحة أن أرواح أقل كلفة إذا المعادلة ا...,لازال الشبيحة أن أرواح أقل كلفة إذا المعادلة ا...
1,امريكان و يا اذنابهم ان امن تستطيعوا لان و ستر...,أمريكان ويا أذنابهم أن أمن تستطيعون لأن وسترون...,أمريكان ويا أذنابهم أن أمن تستطيعون لأن وسترون...,أمريكان ويا أذنابهم أن أمن تستطيعون لأن وسترون...
2,إعتقاد,اعتقاد,اعتقاد,اعتقاد
3,راينا اماكن و للتقطيع بالتاكيد او او ابيا واسف...,رأينا أماكن وللتقطيع بالتأكيد أو أو أبيا وأسفا...,رأينا أماكن وللتقطيع بالتأكيد أو أو أبيا وأسفا...,رأينا أماكن وللتقطيع بالتأكيد أو أو أبيا وأسفا...
4,الى مشاركه انت الاحرار لايخافون لانهم الانفس ا...,إلى مشاركة أنت الأحرار لا يخافون لأنهم الأنفس ...,إلى مشاركة أنت الأحرار لا يخافون لأنهم الأنفس ...,إلى مشاركة أنت الأحرار لا يخافون لأنهم الأنفس ...
6,اخر ايامك احرار الا ان تكبرو ان اتي احرار صلو ...,آخر أيامك أحرار إلا أن تكبروا إن أتى أحرار صلو...,آخر أيامك أحرار إلا أن تكبروا إن أتى أحرار صلو...,آخر أيامك أحرار إلا أن تكبروا إن أتى أحرار صلو...
7,اثبت انه اسد و في اوقف الاسرائيلي سوري فماكان...,أثبت أنه أسد وفي ، أوقف الإسرائيلي سورية فما ك...,أثبت أنه أسد وفي ، أوقف الإسرائيلي سورية فما ك...,أثبت أنه أسد وفي ، أوقف الإسرائيلي سورية فما ك...
8,الاطفال ارهابيون لانهم الامة الامة الى اخر الا...,الأطفال إرهابيون لأنهم الأمة الأمة إلى آخر ألا...,الأطفال إرهابيون لأنهم الأمة الأمة إلى آخر ألا...,الأطفال إرهابيون لأنهم الأمة الأمة إلى آخر ألا...
9,احبكم ارض امام اجندة إحتلال وامام انا إبن إبن,أحبكم أرض أمام أجندة احتلال وأمام أنا ابن ابن,أحبكم أرض أمام أجندة احتلال وأمام أنا ابن ابن,أحبكم أرض أمام أجندة احتلال وأمام أنا ابن ابن
10,؟ اوجاعا و مصائب و تقتلون و تسرقون و تعذبون و ...,، أوجاعا ومصائب وتقتلون وتسرقون وتعذبون وتكذبو...,، أوجاعا ومصائب وتقتلون وتسرقون وتعذبون وتكذبو...,، أوجاعا ومصائب وتقتلون وتسرقون وتعذبون وتكذبو...


In [109]:
df['correction'] = df['correction'].apply(standerdize)


In [110]:
df = df.drop(columns=['correction_old', 'correction_new'])

df.head(10)


,input,correction
0,لا زال الشبيحه ان ارواح اقل كلفه اذا المعادله ...,لازال الشبيحة أن أرواح أقل كلفة إذا المعادلة ا...
1,امريكان و يا اذنابهم ان امن تستطيعوا لان و ستر...,أمريكان ويا أذنابهم أن أمن تستطيعون لأن وسترون...
2,إعتقاد,اعتقاد
3,راينا اماكن و للتقطيع بالتاكيد او او ابيا واسف...,رأينا أماكن وللتقطيع بالتأكيد أو أو أبيا وأسفا...
4,الى مشاركه انت الاحرار لايخافون لانهم الانفس ا...,إلى مشاركة أنت الأحرار لا يخافون لأنهم الأنفس ...
6,اخر ايامك احرار الا ان تكبرو ان اتي احرار صلو ...,آخر أيامك أحرار إلا أن تكبروا إن أتى أحرار صلو...
7,اثبت انه اسد و في اوقف الاسرائيلي سوري فماكان...,أثبت أنه أسد وفي ، أوقف الإسرائيلي سورية فما ك...
8,الاطفال ارهابيون لانهم الامة الامة الى اخر الا...,الأطفال إرهابيون لأنهم الأمة الأمة إلى آخر ألا...
9,احبكم ارض امام اجندة إحتلال وامام انا إبن إبن,أحبكم أرض أمام أجندة احتلال وأمام أنا ابن ابن
10,؟ اوجاعا و مصائب و تقتلون و تسرقون و تعذبون و ...,، أوجاعا ومصائب وتقتلون وتسرقون وتعذبون وتكذبو...
